# Predicción de comportamientos con Word2vec

## Importando Datos

### Extrayendo la secuencia de Comportamientos Colectivos de los Clientes

In [1]:
import pandas as pd
import gensim
from gensim import corpora, models, similarities

In [2]:
# data = pd.read_csv("../resultados/mobile_consume1_results.csv")
data = pd.read_csv("../../resultados/mobile_consume1_results.csv")

In [3]:
# Lista de Clientes
clientes = data.CO_ID.unique()

In [4]:
# Lista de Periodos para la Opcion 1
year_min = min(data.YEAR) 
year_max = max(data.YEAR) 
week_min = min(data.WEEK) 
week_max = max(data.WEEK) 

periodos = []

if year_min == year_max:
    year = year_min
    for week in range(week_min,week_max+1):
        weeks = ""
        if week < 10:
            weeks = "0"+str(week)
        else:
            weeks = str(week)
        periodos.append(str(year)+"-"+weeks)
        
else:
    for year in range(year_min,year_max+1):
        if year == year_min:
            for week in range(week_min,53):
                
                weeks = ""
                if week < 10:
                    weeks = "0"+str(week)
                else:
                    weeks = str(week)
                periodos.append(str(year)+"-"+weeks)
                #print(year,weeks)
                
        elif year == year_max:   
            for week in range(1,week_max+1):
                
                weeks = ""
                if week < 10:
                    weeks = "0"+str(week)
                else:
                    weeks = str(week)
                periodos.append(str(year)+"-"+weeks)
                #print(year,weeks)
                
        else:    
            for week in range(1,53):
            
                weeks = ""
                if week < 10:
                    weeks = "0"+str(week)
                else:
                    weeks = str(week)
                periodos.append(str(year)+"-"+weeks)
                #print(year,weeks)


## Construyendo CORPUS 

### Opción 1 (considerando "SinActividad")

In [142]:
# Construyendo CORPUS CON CADA TEMPORADA DE CLIENTES (considerando las temporadas vacias)

comportamientos=[]
    
for index, cliente in enumerate(clientes):
    lista = []
    for index, periodo in enumerate(periodos):
        lista.append("SinActividad")
        
    comportamientos.append(lista)
    

In [143]:
lista_periodos = list(periodos)

for i in range(len(data)):
    # en la comportamiendo del cliente i
    fila = data.loc[i,['CO_ID','YEAR','WEEK','COLLECTIVE_CLUSTER']]
    # en el indice del cliente i_c  y del periodo i_p 
    index_c = list(clientes).index(int(fila[0]))
    week = ""
    if int(fila[2]) < 10:
        week = "0" + str(int(fila[2]))
    else:
        week = str(int(fila[2]))
    tem = str(int(fila[1]))+"-"+week
    index_p = lista_periodos.index(tem)
        
    try:
        num = ""
        if int(fila[3]) < 10:
            num = "0" + str(int(fila[3]))
        else:
            num = str(int(fila[3]))
        
        comportamientos[index_c][index_p] = "cluster_"+num
    except Exception as e:
        print("Oops!  That was no valid index.  Try again...")
    

In [144]:
tok_corp = comportamientos

### Opción 2 (sin considerar "SinActividad")

In [110]:
tok_corp = []

for cli in clientes:
    data_i = data[(data.CO_ID == cli)]
    lista_i = list(data_i.COLLECTIVE_CLUSTER)

    for j in range(len(lista_i)):
        dat = ""
        if int(str(lista_i[j])) < 10:
            dat = "0" + str(lista_i[j])
        else:
            dat = str(int(fila[2]))
            
        lista_i[j] = "cluster_"+dat
    tok_corp.append(lista_i)
    

In [111]:
tok_corp[5]

['cluster_03',
 'cluster_32',
 'cluster_32',
 'cluster_32',
 'cluster_32',
 'cluster_32',
 'cluster_32',
 'cluster_32',
 'cluster_32',
 'cluster_32',
 'cluster_32',
 'cluster_32']

## Modelo

In [145]:
model = Word2Vec(tok_corp, size=100, window=5, min_count=1, workers=4 , hs=1)
model

In [146]:
#sorted(model.wv.index2word)

In [147]:
vector = model.wv['SinActividad']
len(vector)

100

In [148]:
from gensim.models import KeyedVectors

In [149]:

vector

array([ 0.1814216 , -0.22845167, -0.10052143,  0.20441374, -0.16506666,
        0.18865345, -0.03582837,  0.11229225, -0.11396419,  0.1240678 ,
       -0.12099393,  0.1778392 ,  0.19470225, -0.13662392, -0.23999298,
       -0.07773545,  0.19150737,  0.22434475,  0.1167999 ,  0.117656  ,
       -0.14593594,  0.32835945,  0.0309642 , -0.12255155, -0.10198585,
        0.1075408 ,  0.25217608, -0.34918347,  0.03222382, -0.08883551,
        0.36220634, -0.22203083,  0.01316364, -0.29482996,  0.14828742,
        0.03339225, -0.13712533, -0.40695235,  0.11991318, -0.12607692,
        0.20964563, -0.19365044,  0.2549993 ,  0.26660722, -0.1417003 ,
       -0.1783807 ,  0.06713612, -0.00403908,  0.06734915,  0.10841063,
       -0.24514255, -0.02699099,  0.09539118,  0.12771015,  0.38980094,
       -0.14112061, -0.21478824,  0.07035387, -0.36050978,  0.1021275 ,
       -0.19682233, -0.03278184, -0.40605107, -0.10521816,  0.0897701 ,
        0.13516383, -0.23134322, -0.28735417,  0.07708653, -0.10

In [119]:
model = gensim.models.Word2Vec(tok_corp, min_count=1, window=5, size=7 , workers=4)

In [127]:
model.vocabulary.max_final_vocab

In [70]:
# Diccionario

# sorted(model.wv.index2word)

In [150]:
# VOCABULARIO

print(sorted(list(model.wv.vocab.keys())))
print("Cantidad = "+str(len(model.wv.vocab)))

['SinActividad', 'cluster_00', 'cluster_01', 'cluster_02', 'cluster_03', 'cluster_04', 'cluster_05', 'cluster_06', 'cluster_07', 'cluster_08', 'cluster_09', 'cluster_10', 'cluster_11', 'cluster_12', 'cluster_13', 'cluster_14', 'cluster_15', 'cluster_16', 'cluster_17', 'cluster_18', 'cluster_19', 'cluster_20', 'cluster_21', 'cluster_22', 'cluster_23', 'cluster_24', 'cluster_25', 'cluster_26', 'cluster_27', 'cluster_28', 'cluster_29', 'cluster_30', 'cluster_31', 'cluster_32', 'cluster_33', 'cluster_34', 'cluster_35', 'cluster_36', 'cluster_37', 'cluster_38', 'cluster_39', 'cluster_40', 'cluster_41', 'cluster_42', 'cluster_43', 'cluster_44', 'cluster_45', 'cluster_46', 'cluster_47', 'cluster_48', 'cluster_49', 'cluster_50', 'cluster_51', 'cluster_52', 'cluster_53', 'cluster_54', 'cluster_55', 'cluster_56', 'cluster_57', 'cluster_58', 'cluster_59', 'cluster_60', 'cluster_61', 'cluster_62', 'cluster_63', 'cluster_64', 'cluster_65', 'cluster_66', 'cluster_67', 'cluster_68', 'cluster_69']
Can

### Resultados

In [140]:
model.predict_output_word(["SinActividad","SinActividad"])

/home/jkn/anaconda3/lib/python3.6/site-packages/gensim/models/word2vec.py:1140: UserWarning: All the input context words are out-of-vocabulary for the current model.
  warnings.warn("All the input context words are out-of-vocabulary for the current model.")


In [76]:
model.train([['hello','wordld'],['hello','hello','humf']], total_examples=1, epochs=1)

(0, 5)

In [87]:
model.most_similar("cluster_27")

/home/jkn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/home/jkn/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('cluster_35', 0.873235821723938),
 ('cluster_09', 0.8602551817893982),
 ('cluster_47', 0.8581576347351074),
 ('cluster_41', 0.8390741348266602),
 ('cluster_19', 0.8255566358566284),
 ('cluster_59', 0.8234487175941467),
 ('cluster_39', 0.822681188583374),
 ('cluster_37', 0.7860894799232483),
 ('cluster_12', 0.7354577779769897),
 ('cluster_48', 0.7170480489730835)]

In [88]:
model.most_similar("cluster_35")

/home/jkn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/home/jkn/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('cluster_27', 0.873235821723938),
 ('cluster_39', 0.8504495620727539),
 ('cluster_19', 0.8471742868423462),
 ('cluster_47', 0.844931960105896),
 ('cluster_41', 0.8433172106742859),
 ('cluster_59', 0.8361415863037109),
 ('cluster_09', 0.8259369134902954),
 ('cluster_37', 0.7688354253768921),
 ('cluster_69', 0.7568635940551758),
 ('cluster_12', 0.7543806433677673)]

In [28]:
import random

In [63]:
prueba = []

"""for i in range(3): 
    prueba.append("cluster_"+str(random.randint(0,69))+".0")
    """
prueba = ['cluster_29',"cluster_00", "SinActividad", "SinActividad"]

In [64]:
prueba

['cluster_29', 'cluster_00', 'SinActividad', 'SinActividad']

In [65]:
# Predicion de la siguiente palabra

model.predict_output_word(prueba, topn=5)

[('cluster_00', 0.03985915),
 ('cluster_27', 0.03670628),
 ('cluster_41', 0.03650439),
 ('cluster_09', 0.03147976),
 ('cluster_35', 0.028495274)]

In [56]:
model.predict_output_word(prueba, topn=20)

[('cluster_49', 0.02102846),
 ('cluster_35', 0.020178417),
 ('cluster_15', 0.020018263),
 ('cluster_07', 0.019643178),
 ('cluster_40', 0.019266814),
 ('cluster_25', 0.019191433),
 ('cluster_27', 0.019084098),
 ('cluster_66', 0.018588684),
 ('cluster_56', 0.018498663),
 ('cluster_46', 0.018433262),
 ('cluster_04', 0.017946504),
 ('cluster_26', 0.017863272),
 ('cluster_44', 0.01784515),
 ('cluster_32', 0.01722876),
 ('cluster_55', 0.017153617),
 ('cluster_64', 0.016960876),
 ('cluster_59', 0.016897112),
 ('cluster_47', 0.016843623),
 ('cluster_36', 0.01680604),
 ('cluster_60', 0.016775558)]

In [46]:
# Modelos

model["cluster_00"]

/home/jkn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


array([-4.012762  ,  0.37017253, -3.2437015 ,  0.18191081, -1.0842882 ,
        1.6514736 , -0.7659784 ], dtype=float32)

In [47]:
model["SinActividad"]

/home/jkn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.13033447,  0.2978028 , -0.72173613,  0.21796066, -0.3534182 ,
        0.6917916 ,  0.27593198], dtype=float32)

In [109]:
# Parecidos

model.most_similar(["SinActividad"])

/home/jkn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


KeyError: "word 'SinActividad' not in vocabulary"

In [108]:
model.syn1neg

/home/jkn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn1neg` (Attribute will be removed in 4.0.0, use self.trainables.syn1neg instead).
  """Entry point for launching an IPython kernel.


array([[-0.1037876 , -0.11770578, -0.04433591, ...,  0.13152337,
        -0.11143198,  0.44055644],
       [-0.3243582 ,  0.00965084, -0.08123688, ...,  0.25315443,
         0.02785533,  0.37246758],
       [-0.21043761, -0.07495611, -0.08578566, ...,  0.07919277,
         0.00849154,  0.35399908],
       ...,
       [-0.00323414, -0.17311643, -0.0983289 , ..., -0.08709188,
        -0.22905566,  0.43923855],
       [ 0.04122573, -0.08885468, -0.11941069, ..., -0.06421556,
        -0.26085785,  0.44040588],
       [ 0.01862435, -0.0697745 , -0.12925376, ..., -0.13472837,
        -0.22791705,  0.45035952]], dtype=float32)

In [97]:
model.build_vocab(["cluster_555"])

RuntimeError: cannot sort vocabulary after model weights already initialized.

In [98]:
model.wv.most_similar(["cluster_51"])

KeyError: "word 'cluster_51' not in vocabulary"

### Referencias

In [51]:

# https://github.com/RaRe-Technologies/gensim/tree/develop/docs/notebooks
    